In [40]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from scipy.signal import find_peaks

def get_agar_spot_color(im):
    im_ = np.asarray(im).ravel()
    bincnt = np.bincount(im_)
    peaks, _ = find_peaks(bincnt, width=3)
    vals = bincnt[peaks]
    idx = np.argsort(vals)[::-1]
    color_agar = peaks[idx[0]]
    second_idx = idx[idx > idx[0]]
    if len(second_idx) == 0:
        color_spot = 0.7 * im_.max()
    else:
        color_spot = peaks[second_idx[0]]
    lower_lim_spots = 2 * color_agar
    upper_lim_spots = 0.7 * im_.max()
    if color_spot < lower_lim_spots:
        color_spot = min(lower_lim_spots, upper_lim_spots)
    elif color_spot > upper_lim_spots:
        color_spot = max(lower_lim_spots, upper_lim_spots)
    return int(color_agar), int(color_spot)

def generate_pattern(nrow, ncol, rpix, color_agar, color_spot):
    rspot = int((rpix * 0.5) / 2)
    pattern = np.ones((nrow * rpix, ncol * rpix), dtype=np.uint8) * color_agar
    for i, j in np.ndindex(nrow, ncol):
        cv2.circle(pattern, (j * rpix + int(rpix / 2), i * rpix + int(rpix / 2)), rspot, color_spot, -1)
    return pattern

def show_grid_result(latest_image, min_loc, pat_h, pat_w, nrow, ncol):
    to_show_grid = cv2.imread(latest_image)
    d_h = int(pat_h / nrow)
    d_w = int(pat_w / ncol)
    for i in range(nrow+1):
        for j in range(ncol+1):
            cv2.line(
                to_show_grid, (min_loc[0]+d_w*j, min_loc[1]),
                (min_loc[0]+d_w*j, min_loc[1]+d_h*(nrow)), (255,0,0), 5)
        cv2.line(
            to_show_grid, (min_loc[0], min_loc[1]+d_h*i),
            (min_loc[0]+d_w*(ncol), min_loc[1]+d_h*i), (255,0,0), 5)
    plt.figure(figsize=(8,8))
    plt.imshow(cv2.cvtColor(to_show_grid, cv2.COLOR_BGR2RGB))
    plt.title("Grid overlay result")
    plt.axis('off')
    plt.show()

def interactive_grid_detection(image_path, nrow, ncol, frac, grid_by_peaks=False):
    img_rgb = cv2.imread(image_path)
    img_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    quants = np.quantile(img_gray.ravel(), [0.01, 0.99])
    img_gray = np.clip(img_gray, *quants)
    img_gray = np.array(img_gray, dtype=np.uint8)

    # get agar and spot colors
    im_ = img_gray.ravel()
    bincnt = np.bincount(im_)
    peaks, _ = find_peaks(bincnt, width=3)
    color_agar, color_spot = get_agar_spot_color(img_gray)

    img_gray_clip = np.clip(img_gray, color_agar, color_spot)
    img_gray_clip = np.array(img_gray_clip, dtype=np.uint8)

    # Plot input image, grayscale image, and clipped grayscale image
    plt.figure(figsize=(12,5))
    plt.subplot(1,3,1)
    plt.imshow(cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB))
    plt.title("Input Image (RGB)")
    plt.axis('off')

    plt.subplot(1,3,2)
    plt.imshow(img_gray, cmap='gray')
    plt.title("Grayscale Image")
    plt.axis('off')

    plt.subplot(1,3,3)
    plt.imshow(img_gray_clip, cmap='gray')
    plt.title("Grayscale Image Clipped")
    plt.axis('off')
    plt.show()

    # Plot histogram and peaks
    plt.figure(figsize=(8,4))
    plt.plot(bincnt, label='Histogram')
    plt.plot(peaks, bincnt[peaks], "x", label='Peaks')
    plt.axvline(color_agar, color='g', linestyle='--', label='Agar color')
    plt.axvline(color_spot, color='r', linestyle='--', label='Spot color')
    plt.legend()
    plt.title("Intensity Histogram with Peaks")
    plt.xlabel("Intensity")
    plt.ylabel("Count")
    plt.show()
    print(f"Agar color: {color_agar}, Spot color: {color_spot}")

    if grid_by_peaks:
        # Project image along axes
        y_profile = img_gray.mean(axis=1)
        x_profile = img_gray.mean(axis=0)

        # Find all peaks (no distance constraint)
        y_peaks_all, _ = find_peaks(y_profile, width = 0.1*frac*img_gray.shape[0]//nrow, distance=0.75*frac*img_gray.shape[0]//nrow)
        x_peaks_all, _ = find_peaks(x_profile, width = 0.1*frac*img_gray.shape[0]//ncol, distance=0.75*frac*img_gray.shape[1]//ncol)

        # Take nrow/ncol peaks closest to the center
        center_y = img_gray.shape[0] // 2
        center_x = img_gray.shape[1] // 2

        y_center_idx = np.argsort(np.abs(y_peaks_all - center_y))[:nrow]
        x_center_idx = np.argsort(np.abs(x_peaks_all - center_x))[:ncol]

        y_peaks = np.sort(y_peaks_all[y_center_idx])
        x_peaks = np.sort(x_peaks_all[x_center_idx])

        # Calculate average spacing
        avg_dy = np.mean(np.diff(y_peaks))
        avg_dx = np.mean(np.diff(x_peaks))

        # Grid should start half a spacing before the first peak
        min_y = int(round(y_peaks[0] - 0.5 * avg_dy))
        min_x = int(round(x_peaks[0] - 0.5 * avg_dx))

        # Grid should end half a spacing after the last peak
        pat_h = int(round((y_peaks[-1] - y_peaks[0]) + avg_dy))
        pat_w = int(round((x_peaks[-1] - x_peaks[0]) + avg_dx))
        min_loc = (min_x, min_y)

        # Show detected peaks (all in red, kept in green)
        plt.figure(figsize=(12,4))
        plt.subplot(1,2,1)
        plt.plot(y_profile)
        plt.plot(y_peaks_all, y_profile[y_peaks_all], "rx", label="All peaks")
        plt.plot(y_peaks, y_profile[y_peaks], "gx", label="Kept peaks")
        plt.title("Y-axis mean & peaks")
        plt.legend()
        plt.subplot(1,2,2)
        plt.plot(x_profile)
        plt.plot(x_peaks_all, x_profile[x_peaks_all], "rx", label="All peaks")
        plt.plot(x_peaks, x_profile[x_peaks], "gx", label="Kept peaks")
        plt.title("X-axis mean & peaks")
        plt.legend()
        plt.show()

        # Show grid result using the same function
        show_grid_result(image_path, min_loc, pat_h, pat_w, nrow, ncol)
        return



    h, w = img_gray.shape
    rpix = int(h / nrow)
    pattern = generate_pattern(nrow, ncol, rpix, color_agar, color_spot)
    plt.figure(figsize=(6,6))
    plt.imshow(pattern, cmap='gray')
    plt.title("Generated Pattern")
    plt.axis('off')
    plt.show()

    min_fraction, max_fraction = frac, 1.0
    iterations = int((max_fraction - min_fraction) * 100 / 0.2)
    found = None
    best_res = None
    for idx, fraction in enumerate(np.linspace(min_fraction, max_fraction, iterations)):
        pat_w = int(w * fraction)
        pat_h = int(pat_w * nrow / ncol)
        if pat_h > h:
            pat_h = int(h * fraction)
            pat_w = int(pat_h * ncol / nrow)
        pattern_scaled = cv2.resize(pattern, (pat_w, pat_h))
        res = cv2.matchTemplate(img_gray, pattern_scaled, cv2.TM_SQDIFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        if found is None or min_val < found[0]:
            found = (min_val, min_loc, pat_h, pat_w)
            best_res = res
        if idx in [0, iterations//2, iterations-1]:
            plt.figure(figsize=(6,6))
            plt.imshow(res, cmap='hot')
            plt.title(f"matchTemplate result (fraction={fraction:.2f})")
            plt.colorbar()
            plt.show()
    print("cv2.matchTemplate: slides the pattern over the image, computes a similarity score at each position. With TM_SQDIFF_NORMED, lower is better.")
    print("cv2.minMaxLoc: finds the location of the minimum (best) value in the result map.")

    min_val, min_loc, pat_h, pat_w = found
    print(f"Best match: min_val={min_val}, min_loc={min_loc}, pat_h={pat_h}, pat_w={pat_w}")
    show_grid_result(image_path, min_loc, pat_h, pat_w, nrow, ncol)

interact(
    interactive_grid_detection,
    image_path=widgets.Text(value='F://Problemos/ST2PRE.3-ST59.1_R1/Images/QFA1755171763_2025-08-07_11-40-53-grid.jpg', description='Image Path:'),
    nrow=widgets.IntSlider(value=8, min=1, max=24, step=1, description='Rows:'),
    ncol=widgets.IntSlider(value=12, min=1, max=24, step=1, description='Cols:'),
    frac=widgets.FloatSlider(value=0.8, min=0.1, max=1.0, step=0.05, description='Frac:'),
    grid_by_peaks=widgets.Checkbox(value=True, description='Grid by Peaks')
)

interactive(children=(Text(value='F://Problemos/ST2PRE.3-ST59.1_R1/Images/QFA1755171763_2025-08-07_11-40-53-gr…

<function __main__.interactive_grid_detection(image_path, nrow, ncol, frac, grid_by_peaks=False)>